In [ ]:
import cv2
import IPython.display 
import importlib
import skimage.io as imgio
import numpy as np
import os
import torch
import sys

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
    
def showCube(vol):
    v1=np.max(vol,axis=0)
    v2=np.max(vol,axis=1)
    v3=np.max(vol,axis=2)
    imshow(v1*255)
    imshow(v2*255)
    imshow(v3*255)

def showCubes(vols):
    v1s=[]
    v2s=[]
    v3s=[]
    for v in vols:
        v1s.append(np.max(v,axis=0))
        v2s.append(np.max(v,axis=1))
        v3s.append(np.max(v,axis=2))
    imshow(np.concatenate(v1s,1)*255)
    imshow(np.concatenate(v2s,1)*255)
    imshow(np.concatenate(v3s,1)*255)

In [ ]:
import sys
sys.path.append("./")
import run_v2

In [ ]:
tl=iter(run_v2.trn.dataLoader)
img,lbl=tl.next()
i=img[0,0]
i-=i.min()
i/=i.max()
i0,_=i.max(0)
i0=i0*255
i1,_=i.max(1)
i1=i1*255
i2,_=i.max(2)
i2=i2*255
#showCube(i.data.cpu().numpy())
l=lbl[0].clone()
print(np.unique(l))
# remap 255->0 0->1 1->2
# rescale to 0-255
l[l==255]=-1
l=(l+1)*127
l0,_=l.max(0)
l1,_=l.max(1)
l2,_=l.max(2)
imshow(i0.numpy())
imshow(i1.numpy())
imshow(i2.numpy())
imshow(l0.numpy())
imshow(l1.numpy())
imshow(l2.numpy())
#imshow(np.equal(l0.numpy(),255)*255)
#imshow(np.equal(l0.numpy(),1)*255)
imshow(torch.stack([torch.zeros_like(i0),i0,l0.float(),],dim=-1).numpy())
imshow(torch.stack([torch.zeros_like(i1),i1,l1.float(),],dim=-1).numpy())
imshow(torch.stack([torch.zeros_like(i2),i2,l2.float(),],dim=-1).numpy())

In [ ]:
out=run_v2.net.forward(img.cuda())

In [ ]:
np.unique(out[0][1].data.cpu().numpy())

In [ ]:
o=out[0][1].data.cpu().numpy()
print(o.shape)
o-=np.min(o)
o/=np.max(o)
showCube(o)

In [ ]:
l=run_v2.loss
c=l.forward(out,lbl.cuda())
out.retain_grad()

In [ ]:
c.backward()

In [ ]:
g=np.abs(out.grad[0][1].data.cpu().numpy())
print(g.shape)
g-=np.min(g)
g/=np.max(g)
showCube(g)